In [130]:
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
import warnings
warnings.filterwarnings('ignore')
import pingouin as pg
import statsmodels.api as sm

In [56]:
data = pd.read_pickle('data99.pkl')
data_use = data.copy()

In [57]:
#########################
### Table 1 Panel B #####
#########################

factors = ['MV', 'BV', 'SV', 'MG', 'BG', 'SG']

for factor in factors:
    data_use[f'{factor}_me'] = data_use[factor] * data_use['me']
grouped = data_use.groupby('jdate')[[f'{factor}_me' for factor in factors]].sum()
grouped['total_me'] = data_use.groupby('jdate')['me'].sum()
for factor in factors:
    grouped[factor] = grouped[f'{factor}_me'] / grouped['total_me']
avg_market_cap_percentage = grouped[factors].mean() * 100
avg_market_cap_percentage = avg_market_cap_percentage.round(1)

table1_panelB = pd.DataFrame([avg_market_cap_percentage.values], columns=factors)
table1_panelB.index = ['Average percent of MC']
table1_panelB

,MV,BV,SV,MG,BG,SG
Average percent of MC,14.0,11.7,2.3,51.1,47.5,3.6


In [77]:
#########################
### Table 1 Panel A #####
#########################
data_use = data.copy()

before = data_use[(data_use['jdate']>='1963-07-31') & (data_use['jdate']<='1991-06-30')]
after = data_use[(data_use['jdate']>'1991-06-30') & (data_use['jdate']<='2019-06-30')]
whole = data_use.copy()

In [96]:
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return np.nan

def calculate_portfolio_returns(data, portfolios,var,weight):
    result = pd.DataFrame()

    for portfolio in portfolios:
        subset = data[data[portfolio] == 1]
        result[portfolio] = subset.groupby('jdate').apply(wavg, var, weight)
    result['all_vwret'] = data.groupby('jdate').apply(wavg, var, weight)

    for portfolio in portfolios:
        result[portfolio + '_excess'] = result[portfolio] - result['all_vwret']

    portfolio_return = result.reset_index()
    return portfolio_return

portfolios = ['MV', 'BV', 'SV', 'MG', 'BG', 'SG']

portfolio_return = calculate_portfolio_returns(whole,portfolios,'ret_ahead','me')

In [98]:
portfolio_return.to_pickle('portfolio_return.pkl')

In [62]:
### 尽管这种方法在计量上更加合理，但是算出来的结果似乎并不是非常nice......

# portfolio_return00 = portfolio_return.copy()
# X = sm.add_constant(portfolio_return00['post_dummy'])  
# y = portfolio_return00['SV_excess'] 
# model = sm.OLS(y, X).fit(HAC_cov_type='HAC', cov_kwds={'maxlags': 5}) 
# result = pd.DataFrame()
# mv_coefficient = model.params['post_dummy']  # 系数
# std_err = model.bse['post_dummy']
# std_dev = portfolio_return00['SV_excess'].std(ddof=0)
# t_stat = mv_coefficient / std_err  # t值

# # 构建竖着排列的 DataFrame
# result = pd.DataFrame({
#     'Metric': ['Value', 'std_dev', 'TStat'],
#     'MV': [mv_coefficient * 100, std_dev * 100, t_stat]
# })

In [101]:
portfolio_return00 = portfolio_return.copy()
portfolio_return00['post_dummy'] = (portfolio_return00['jdate'] <= '1991-06-30').astype(int)
subset = portfolio_return00.copy()
subset00 = portfolio_return00[portfolio_return00['post_dummy']==1].reset_index(drop=True)
subset11 = portfolio_return00[portfolio_return00['post_dummy']==0].reset_index(drop=True)
subset99 = subset00 - subset11

In [102]:
def summary(data, portfolio):
    portfolio_name = portfolio + '_excess'
    _data00 = data[portfolio_name]
    _data00 = _data00

    mean = _data00.mean()  
    var = _data00.var(ddof=1)  
    sd = var ** 0.5  
    n = len(_data00)  
    se = (var / n) ** 0.5 
    t_stat_diff = mean / se 

    result = pd.DataFrame(
        [
            mean ,  
            sd ,   
            t_stat_diff  
        ],
        index=['Average Excess Return', 'Standard Deviation', 't-Statistic'],
        columns=[portfolio]
    )

    return result

In [104]:
result00 = pd.concat(
    [summary(subset, portfolio=portfolio) for portfolio in portfolios], 
    axis=1
).round(2)

result11 = pd.concat(
    [summary(subset00, portfolio=portfolio) for portfolio in portfolios], 
    axis=1
).round(2)

result22 = pd.concat(
    [summary(subset11, portfolio=portfolio) for portfolio in portfolios], 
    axis=1
).round(2)

result33 = pd.concat(
    [summary(subset99, portfolio=portfolio) for portfolio in portfolios], 
    axis=1
).round(2)

table1_panelA = pd.concat([result00,result11,result22,result33], axis=0)
table1_panelA

,MV,BV,SV,MG,BG,SG
Average Excess Return,0.26,0.19,0.49,-0.03,-0.02,0.08
Standard Deviation,2.70,2.75,3.64,1.12,1.17,3.61
t-Statistic,2.53,1.81,3.49,-0.63,-0.53,0.60
Average Excess Return,0.41,0.33,0.61,-0.04,-0.04,0.16
Standard Deviation,2.32,2.24,3.30,1.25,1.29,3.42
t-Statistic,3.22,2.74,3.39,-0.62,-0.60,0.84
Average Excess Return,0.12,0.05,0.37,-0.01,-0.01,0.01
Standard Deviation,3.03,3.17,3.95,0.98,1.03,3.80
t-Statistic,0.72,0.28,1.71,-0.24,-0.10,0.04
Average Excess Return,0.29,0.29,0.24,-0.03,-0.04,0.15


In [105]:
portfolio_return11 = portfolio_return.copy()

In [ ]:
#########################
### Table 1 Panel C #####
#########################

variables = ['all_vwret', 'MV', 'BV', 'SV', 'MG', 'BG', 'SG']
subset = portfolio_return11[variables] 
corr_matrix = subset.corr() 
corr_matrix00 = pd.DataFrame(corr_matrix.iloc[0, 1:].round(2)).T 

variables = ['MV_excess', 'BV_excess', 'SV_excess', 'MG_excess', 'BG_excess', 'SG_excess']
subset = portfolio_return11[variables]
corr_matrix11 = subset.corr().round(2)  
corr_matrix11.columns = corr_matrix11.columns.str.replace('_excess', '')  
corr_matrix11.index = corr_matrix11.index.str.replace('_excess', '') 

corr_matrix = pd.concat([corr_matrix00, corr_matrix11])
corr_matrix.index.values[0] = 'Cor(R, RM)' 
table1_panelC = corr_matrix

In [137]:
#########################
###  Hotelling T^2  #####
#########################
X = subset00[variables]
Y = subset11[variables]
print(pg.multivariate_ttest(X, Y,paired=False))

                 T2        F  df1  df2      pval
hotelling  4.029551  0.66658    6  665  0.676747


In [138]:
print(table1_panelA)
print(pg.multivariate_ttest(X, Y,paired=False))
print(table1_panelB)
print(table1_panelC)

                         MV    BV    SV    MG    BG    SG
Average Excess Return  0.26  0.19  0.49 -0.03 -0.02  0.08
Standard Deviation     2.70  2.75  3.64  1.12  1.17  3.61
t-Statistic            2.53  1.81  3.49 -0.63 -0.53  0.60
Average Excess Return  0.41  0.33  0.61 -0.04 -0.04  0.16
Standard Deviation     2.32  2.24  3.30  1.25  1.29  3.42
t-Statistic            3.22  2.74  3.39 -0.62 -0.60  0.84
Average Excess Return  0.12  0.05  0.37 -0.01 -0.01  0.01
Standard Deviation     3.03  3.17  3.95  0.98  1.03  3.80
t-Statistic            0.72  0.28  1.71 -0.24 -0.10  0.04
Average Excess Return  0.29  0.29  0.24 -0.03 -0.04  0.15
Standard Deviation     3.75  3.76  5.01  1.60  1.68  4.98
t-Statistic            1.41  1.39  0.89 -0.34 -0.39  0.55
                 T2        F  df1  df2      pval
hotelling  4.029551  0.66658    6  665  0.676747
                         MV    BV   SV    MG    BG   SG
Average percent of MC  14.0  11.7  2.3  51.1  47.5  3.6
              MV    BV    SV    MG  